# Embeddings & Training

In [1]:
import pandas as pd
import src.article_relevance as ar

/Users/sedv8808/HT-Data/UWisc/article-relevance/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
publicationDF = pd.read_parquet('data/parquet/neotomaMetadata.parquet')
annotationDF = pd.read_parquet('data/parquet/neotomaAnnotation.parquet')
publicationDF.head(3)

,ARCTIC,Acta Palaeontologica Polonica,Advanced Materials,Advances in Dermatology and Allergology,Aerobiologia,Aerospace Engineering,African Invertebrates,Agricultural and Biological Sciences (miscellaneous),"Agriculture, Ecosystems &amp; Environment",Agronomy and Crop Science,...,iScience,language,published,publisher,subtitle,title,titleSubtitleAbstract,validForPrediction,Écologie et paléoécologie végétale,Écoscience
0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0,...,0.0,en,"{'date-parts': [[1999, 3, 25]]}",Canadian Science Publishing,,Age verification of the Lake Gribben forest be...,age verification of the lake gribben forest be...,1,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0,...,0.0,en,"{'date-parts': [[2008, 6, 28]]}",Wiley,,"Postglacial climate and vegetation history, no...","postglacial climate and vegetation history, no...",1,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0,...,0.0,en,"{'date-parts': [[2007, 11, 23]]}",Consortium Erudit,,Late Quaternary Marine and Terrestrial Environ...,late quaternary marine and terrestrial environ...,1,0.0,0.0


In [3]:
publicationDF.loc[1,'titleSubtitleAbstract']

'postglacial climate and vegetation history, north-central kola peninsula, russia: pollen and diatom records from lake yarnyshnoe-3  '

In [4]:
smallDF = ar.addEmbeddings(publicationDF.head(2), 'titleSubtitleAbstract')
smallDF

2023-09-11 13:29:48,808 - addEmbeddings.py:19 - addEmbeddings - INFO - Sentence embedding start.


No sentence-transformers model found with name /Users/sedv8808/.cache/torch/sentence_transformers/allenai_specter2. Creating a new one with MEAN pooling.


2023-09-11 13:29:53,194 - addEmbeddings.py:37 - addEmbeddings - INFO - Sentence embedding completed.


,ARCTIC,Acta Palaeontologica Polonica,Advanced Materials,Advances in Dermatology and Allergology,Aerobiologia,Aerospace Engineering,African Invertebrates,Agricultural and Biological Sciences (miscellaneous),"Agriculture, Ecosystems &amp; Environment",Agronomy and Crop Science,...,758,759,760,761,762,763,764,765,766,767
0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0,...,-0.317891,-0.165364,-0.144835,-1.041570,-0.354374,-0.547732,-0.443002,0.493709,-0.050640,-0.369954
1,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0,...,-0.148502,-0.208324,0.255299,-1.220001,-0.469716,-0.856498,-0.014106,0.421403,-0.260212,-0.618345


In [5]:
bigDF = ar.addEmbeddings(publicationDF, 'titleSubtitleAbstract')

2023-09-11 13:30:02,967 - addEmbeddings.py:19 - addEmbeddings - INFO - Sentence embedding start.


No sentence-transformers model found with name /Users/sedv8808/.cache/torch/sentence_transformers/allenai_specter2. Creating a new one with MEAN pooling.


2023-09-11 13:38:39,879 - addEmbeddings.py:37 - addEmbeddings - INFO - Sentence embedding completed.


In [6]:
bigDF.columns

Index(['ARCTIC', 'Acta Palaeontologica Polonica', 'Advanced Materials',
       'Advances in Dermatology and Allergology', 'Aerobiologia',
       'Aerospace Engineering', 'African Invertebrates',
       'Agricultural and Biological Sciences (miscellaneous)',
       'Agriculture, Ecosystems &amp; Environment',
       'Agronomy and Crop Science',
       ...
       '758', '759', '760', '761', '762', '763', '764', '765', '766', '767'],
      dtype='object', length=1279)

In [7]:
annotationDF.columns

Index(['index', 'DOI', 'annotation', 'annotator', 'annotationDate'], dtype='object')

In [12]:
annotationDF['annotation'].value_counts()

annotation
Neotoma          6904
Not Neotoma       471
Maybe Neotoma      12
Name: count, dtype: int64

In [8]:
completeData = bigDF.merge(annotationDF, on = 'DOI')
completeData

,ARCTIC,Acta Palaeontologica Polonica,Advanced Materials,Advances in Dermatology and Allergology,Aerobiologia,Aerospace Engineering,African Invertebrates,Agricultural and Biological Sciences (miscellaneous),"Agriculture, Ecosystems &amp; Environment",Agronomy and Crop Science,...,762,763,764,765,766,767,index,annotation,annotator,annotationDate
0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0,...,-0.354374,-0.547732,-0.443002,0.493709,-0.050640,-0.369954,5618,Neotoma,Simon J. Goring,2023-09-01 11:34:37
1,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0,...,-0.354374,-0.547732,-0.443002,0.493709,-0.050640,-0.369954,5619,Neotoma,Simon J. Goring,2023-09-01 11:34:37
2,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0,...,-0.469716,-0.856498,-0.014106,0.421403,-0.260212,-0.618345,6516,Neotoma,Simon J. Goring,2023-09-01 11:34:37
3,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0,...,-0.469716,-0.856498,-0.014106,0.421403,-0.260212,-0.618345,6517,Neotoma,Simon J. Goring,2023-09-01 11:34:37
4,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0,...,-0.025042,-0.923761,0.111560,-0.051684,-0.157385,-0.168050,969,Neotoma,Simon J. Goring,2023-09-01 11:34:37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5882,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,3717,Neotoma,Simon J. Goring,2023-09-01 11:34:37
5883,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,5957,Neotoma,Simon J. Goring,2023-09-01 11:34:37
5884,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,254,Not Neotoma,SimonGoring,2023-05-11 18:54:58.457895-04:00
5885,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,429,Not Neotoma,SimonGoring,2023-05-11 18:10:24.523574-04:00


In [10]:
completeData['annotation'].value_counts()

annotation
Neotoma          5430
Not Neotoma       445
Maybe Neotoma      12
Name: count, dtype: int64

In [13]:
completeData.loc[(completeData['annotation']!= 'Neotoma'), 'target'] = 0
completeData.loc[(completeData['annotation']== 'Neotoma'), 'target'] = 1

/var/folders/9k/yl_4hbsx18x4t12kqpmmd2zw0000gn/T/ipykernel_22513/4027474337.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  completeData.loc[(completeData['annotation']!= 'Neotoma'), 'target'] = 0


In [14]:
completeData = completeData[completeData['validForPrediction'] == 1]
completeData.head(1)

,ARCTIC,Acta Palaeontologica Polonica,Advanced Materials,Advances in Dermatology and Allergology,Aerobiologia,Aerospace Engineering,African Invertebrates,Agricultural and Biological Sciences (miscellaneous),"Agriculture, Ecosystems &amp; Environment",Agronomy and Crop Science,...,763,764,765,766,767,index,annotation,annotator,annotationDate,target
0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0,...,-0.547732,-0.443002,0.493709,-0.05064,-0.369954,5618,Neotoma,Simon J. Goring,2023-09-01 11:34:37,1.0


In [27]:
annotationDF.columns

Index(['index', 'DOI', 'annotation', 'annotator', 'annotationDate'], dtype='object')

In [15]:
X = completeData.drop(columns=['DOI', 'title', 'subtitle', 'author', 'abstract',
       'language',  'URL', 'published', 'CrossRefQueryDate', 'validForPrediction', 
       'titleSubtitleAbstract', 'target', 'annotation', 'annotator', 'annotationDate', 'index'])
y = completeData['target']
# author might lead to bias 
# we are only considering english text, so drop language
# title subtitle and abstract have already been used with the embeddings
# does it matter when it was published
# must be used subject, container-title(journal), 'publisher'

In [16]:
X.head(3)

,ARCTIC,Acta Palaeontologica Polonica,Advanced Materials,Advances in Dermatology and Allergology,Aerobiologia,Aerospace Engineering,African Invertebrates,Agricultural and Biological Sciences (miscellaneous),"Agriculture, Ecosystems &amp; Environment",Agronomy and Crop Science,...,758,759,760,761,762,763,764,765,766,767
0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0,...,-0.317891,-0.165364,-0.144835,-1.041570,-0.354374,-0.547732,-0.443002,0.493709,-0.050640,-0.369954
1,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0,...,-0.317891,-0.165364,-0.144835,-1.041570,-0.354374,-0.547732,-0.443002,0.493709,-0.050640,-0.369954
2,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0,...,-0.148502,-0.208324,0.255299,-1.220001,-0.469716,-0.856498,-0.014106,0.421403,-0.260212,-0.618345


In [31]:
import numpy as np

# Remove columns when the value is less than 5
numericColumns = X.select_dtypes(include=['int', 'float']).columns

# Calculate the sum of '1' values in each numeric column
column_sums_abs = X[numericColumns].abs().sum(axis=0)
column_sums_abs

ARCTIC                                        1.000000
Acta Palaeontologica Polonica                 1.000000
Advanced Materials                            1.000000
Advances in Dermatology and Allergology       1.000000
Aerobiologia                                  1.000000
                                              ...     
763                                        4359.315430
764                                        1729.491699
765                                        1511.837646
766                                        1825.136719
767                                        4450.549805
Length: 1267, dtype: float64

## Feature selection

``` markdown
Already encoded:
['subject', 'container-title', 'publisher']

Do we need to know if an article had an abstract? 
Not for now

Remove subjects or container-titles if they are only used once.
```

In [38]:
X.shape[0]

5779

the preprocessing for subject should be different, instead of joining everything, we should be separating the lists and doing dummy variables

## Start the Pipeline

In [70]:
from sentence_transformers import SentenceTransformer
import datetime
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import LogisticRegression 
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import roc_curve
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

In [73]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [40]:
print(X.columns.to_list())

['ARCTIC', 'Acta Palaeontologica Polonica', 'Advanced Materials', 'Advances in Dermatology and Allergology', 'Aerobiologia', 'Aerospace Engineering', 'African Invertebrates', 'Agricultural and Biological Sciences (miscellaneous)', 'Agriculture, Ecosystems &amp; Environment', 'Agronomy and Crop Science', 'Allergologia et Immunopathologia', 'Allergologie select', 'Allergology International', 'Allergy', 'Allergy, Asthma &amp; Immunology Research', 'American Antiquity', 'American Journal of Botany', 'American Journal of Human Biology', 'American Journal of Plant Sciences', 'American Journal of Respiratory and Critical Care Medicine', 'American Journal of Science', 'American Midland Naturalist', 'Analytical Chemistry', 'Anatomy', 'Animal Science and Zoology', 'Animals', 'Annals of Botany', 'Annals of The Tohoku Geographycal Asocciation', 'Annals of the American Thoracic Society', 'Anthropology', 'Antioxidants', 'Antiquity', 'AoB PLANTS', 'Applications in Plant Sciences', 'Applied Microbiolo

In [87]:
strFeature = ['publisher']
strTransformer = OneHotEncoder(sparse_output=False, handle_unknown='ignore', drop='first')

In [88]:
preprocessor = ColumnTransformer(
        transformers = [
            ("str_preprocessor", strTransformer, strFeature)
        ],
        remainder = "passthrough"
    )

In [89]:
logistic_regression = LogisticRegression()

In [90]:
logreg_model = make_pipeline(preprocessor, 
                             logistic_regression)

In [91]:
param_grid = {
    'logisticregression__C': [0.001, 0.01, 0.1, 1, 10],
    'logisticregression__max_iter': [100, 1000, 10000],
    'logisticregression__penalty': ['l1', 'l2'],
    'logisticregression__solver': ['liblinear', 'lbfgs']
}


In [92]:
# Create a randomized search with cross-validation
randomized_search = RandomizedSearchCV(
    logreg_model,  # Use the pipeline
    param_distributions=param_grid,  # Hyperparameter grid
    n_iter=10,  # Number of random combinations to try
    cv=5,  # Cross-validation folds
    random_state=42  # Random seed for reproducibility
)

In [93]:
# Fit the randomized search to the data
randomized_search.fit(X_train, y_train)

/Users/sedv8808/HT-Data/UWisc/article-relevance/venv/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:227: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/sedv8808/HT-Data/UWisc/article-relevance/venv/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:227: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/sedv8808/HT-Data/UWisc/article-relevance/venv/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:227: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Users/sedv8808/HT-Data/UWisc/article-relevance/venv/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:227: UserWarning: Found unknown categories in columns [0] during transform. These unknown c

RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('columntransformer',
                                              ColumnTransformer(remainder='passthrough',
                                                                transformers=[('str_preprocessor',
                                                                               OneHotEncoder(drop='first',
                                                                                             handle_unknown='ignore',
                                                                                             sparse_output=False),
                                                                               ['publisher'])])),
                                             ('logisticregression',
                                              LogisticRegression())]),
                   param_distributions={'logisticregression__C': [0.001, 0.01,
                                                                  0.1, 1, 10],
                                        'logisticregression__max_iter': [100,
                                                                         1000,
                                                                         10000],
                                        'logisticregression__penalty': ['l1',
                                                                        'l2'],
                                        'logisticregression__solver': ['liblinear',
                                                                       'lbfgs']},
                   random_state=42)

In [94]:
best_pipeline = randomized_search.best_estimator_
best_params = randomized_search.best_params_

In [95]:
best_pipeline

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('str_preprocessor',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore',
                                                                sparse_output=False),
                                                  ['publisher'])])),
                ('logisticregression',
                 LogisticRegression(C=1, penalty='l1', solver='liblinear'))])

In [96]:
best_params

{'logisticregression__solver': 'liblinear',
 'logisticregression__penalty': 'l1',
 'logisticregression__max_iter': 100,
 'logisticregression__C': 1}

In [97]:
accuracy = best_pipeline.score(X_test, y_test)
print(f"Test Accuracy: {accuracy:.4f}")

Test Accuracy: 0.9948


/Users/sedv8808/HT-Data/UWisc/article-relevance/venv/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:227: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [84]:
X_train

,ARCTIC,Acta Palaeontologica Polonica,Advanced Materials,Advances in Dermatology and Allergology,Aerobiologia,Aerospace Engineering,African Invertebrates,Agricultural and Biological Sciences (miscellaneous),"Agriculture, Ecosystems &amp; Environment",Agronomy and Crop Science,...,758,759,760,761,762,763,764,765,766,767
4194,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0,...,-0.338689,-0.347165,0.797741,-1.055909,-0.821595,-0.828851,-0.195298,0.070886,-0.373744,-0.542395
387,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0,...,-0.192040,-0.262531,0.027150,-0.922231,-0.429774,-0.039190,-0.402740,0.280943,-0.613066,-0.738637
4378,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0,...,0.320226,0.059141,0.081104,-1.136563,-0.412522,-0.896613,-0.532158,-0.523298,0.123110,-0.702659
3406,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0,...,-0.159493,0.218024,0.076526,-1.625507,-0.223066,-0.426433,-0.456811,0.251931,-0.854245,-0.905137
2342,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0,...,-0.308670,-0.357825,-0.054747,-0.655132,-0.670960,-0.818340,-0.305119,0.119209,-0.185524,-0.909093
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3772,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0,...,-0.292878,0.212374,0.229548,-0.716126,-0.581884,-0.698562,-0.538683,0.443013,0.284219,-0.548976
5191,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0,...,0.073166,0.281051,0.382895,-1.555885,-0.208163,-1.142046,-0.389735,0.523589,-0.773097,-1.018335
5226,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0,...,0.073166,0.281051,0.382895,-1.555885,-0.208163,-1.142046,-0.389735,0.523589,-0.773097,-1.018335
5390,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0,...,0.155251,0.201325,0.598681,-0.743954,-0.275531,-0.123036,-0.151690,-0.531009,-0.307775,-0.848467
